In [356]:
import pandas as pd

# Code Structures

In [357]:
days_of_the_week = ['Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [358]:
locations = {}
times_to_sites = {}

class Location():
    def __init__(self, name, district, ideal_num_sites = None):
        self.name = name
        self.sites = []
        self.ideal_num_sites = ideal_num_sites
        self.district = district
        locations[name] = self
    def add_site(self, time):
        new_site = Site(self.name, time, self.district)
        self.sites.append(new_site)
    def num_sites(self):
        return len(self.sites)
    def num_site_leaders(self):
        return len([site for site in self.sites if site.has_site_leader])
    def num_full_sites(self): #is a site with 4 people considered full?, add a needs_more_people method()
        pass

class Site:
    def __init__(self, name, time, district):
        self.name = name #location name - aka Harding NOT Harding C
        self.time = time
        self.district = district
        self.members = [] #includes Site Leader
        self.has_site_leader = False
        self.has_driver = False
        if time in times_to_sites.keys():
            times_to_sites[time] += [self]
        else:
            times_to_sites[time] = [self]

    def add_member(self, person): #assumes that the member added is the right type
        self.members.append(person)
        person.assigned_site = self
        self.update_booleans()

    def remove_member(self, person):
        self.members.remove(person)
        person.assigned_site = None
        self.update_booleans()

    def num_staff(self):
        num_staff = len([member for member in self.members if member.in_staff])
        return num_staff

    def num_nonstaff(self):
        num_nonstaff = len([member for member in self.members if not member.in_staff])
        return num_nonstaff

    def update_booleans(self):
        self.has_site_leader = any([person for person in self.members if person.leads_site])
        self.has_driver = any([person for person in self.members if person.drives])
        
    def get_driver_name(self):
        return [person for person in self.members if person.drives][0].name

    def get_non_SL_staff_name(self):
        if self.num_staff() == 2:
            return [person for person in self.members \
                if person.in_staff and not person.leads_site][0]
    
    def get_nonstaff_names(self):
        return [member.name for member in self.members if not member.in_staff]
        
    def clear(self):
        self.members = []
        self.has_site_leader = False
        self.has_driver = False
        return self

def clear_all_sites():
    for time in times_to_sites.keys():
        times_to_sites[time] = [time.clear() for time in times_to_sites[time]]


In [359]:
names_to_site_leaders = {}
names_to_staff_members = {}
names_to_nonstaff = {}
class DecalMember:
    def __init__(self, name, can_drive, availabilities =[]):
        self.name = name
        self.drives = can_drive
        self.in_staff = False
        self.leads_site = False
        self.availabilities = availabilities
        self.assigned_site = None
        self.add_to_record()
    def add_to_record(self):
        names_to_nonstaff[self.name] = self
    def add_availabilities(self, availabilities):
        assert type(availabilities) is dict
        self.availabilities.update(availabilities)

class StaffMember(DecalMember):
    def __init__(self, name, can_drive, availabilities=[]):
        super().__init__(name, can_drive, availabilities)
        self.in_staff = True
    def add_to_record(self):
        names_to_staff_members[self.name] = self

class SiteLeader(StaffMember):
    def __init__(self, name, can_drive, availabilities=[]):
        super().__init__(name, can_drive, availabilities)
        self.in_staff = True
        self.leads_site = True
    def add_to_record(self):
        names_to_site_leaders[self.name] = self

import re
def standardize_time(time):
    time = re.sub(r"11-12pm", "11:00am-12:00pm", time)
    time = re.sub(r"12-1pm", "12:00-1:00pm", time)
    time = re.sub(r"1-2pm", "1:00-2:00pm", time)
    time = re.sub(r"2-3pm", "2:00-3:00pm", time)
    time = re.sub(r"3-4pm", "3:00-4:00pm", time)
    time = re.sub(r"4-5pm", "4:00-5:00pm", time)
    time = re.sub(r"5-6pm", "5:00-6:00pm", time)
    time = re.sub(r"am", "AM", time)
    time = re.sub(r"pm", "PM", time)
    return time

def extract_times(text, sep = ","):
    if ',' not in text:
        times = [text]
    else:
        times = text.split(sep = sep)
    times = [standardize_time(time.strip()) for time in times]
    return times

def order_by_availabilities(lst_of_people):
    ordered = sorted(lst_of_people, key = lambda x: len(x.availabilities))
    return ordered

# Stage 0: Read dataframes

In [360]:
school_availabilities = pd.read_excel('school_availability.xlsx')
sl_availabilities = pd.read_excel('sl_availability_UPDATED.xlsx', header = 0)

# Stage 1: Fix dataframes

### Fix school availabilities

In [361]:
# school_availabilities = school_availabilities.dropna(axis = 'index', how = 'all')

In [362]:
# for row in school_availabilities.index.tolist():
#     for column in days_of_the_week:
#         try:
#             school_availabilities.loc[row, column] = standardize_time(school_availabilities.loc[row, column])
#         except: #NaN values
#             pass
# school_availabilities

In [363]:
# school_availabilities.to_excel('school_availability.xlsx', index = False)

### Fix Site Leader Availabilities

In [364]:
# sl_availabilities.head(3)

In [365]:
# def replace_column_names(df, substrings, replacements):
#     old_column_names = []
#     for substring in substrings:
#         old_column_name = [column for column in list(df.columns) if substring in column][0]
#         old_column_names.append(old_column_name)
#     replacement_dictionary = dict(zip(old_column_names, replacements))
#     df = df.rename(columns = replacement_dictionary)
#     return df
# spreadsheet = replace_column_names(sl_availabilities, ['access', 'available'], ['Drives', 'Times Available'])

In [366]:
# for row in sl_availabilities.index.tolist():
#     available = sl_availabilities.loc[row, 'Times Available']
#     sl_availabilities.loc[row, 'Times Available'] = available.replace('Friday 2-3, 3-4, or 4-5pm - if any of these times work', 'Friday 2:00-3:00pm, Friday 3:00-4:00pm, Friday 4:00-5:00pm')

In [367]:
# for row in sl_availabilities.index.tolist():
#     sl_availabilities.loc[row, 'Times Available'] = standardize_time(sl_availabilities, 'Times Available')

In [368]:
# sl_availabilities.to_excel('sl_availability_UPDATED.xlsx', index = False)

# Stage 2: Test Site Leader Arrangements

### Get all possible sites/locations

In [369]:
for row in school_availabilities.index.tolist():
    school_name = school_availabilities.loc[row, 'School']
    district = school_availabilities.loc[row, 'District']
    ideal_num_sites = school_availabilities.loc[row, 'Number of Sites (provisionally)']
    new_location = Location(school_name, district, ideal_num_sites)
    for day in days_of_the_week:
        times = school_availabilities.loc[row, day]
        if pd.notnull(times):
            times = extract_times(times)
            times = [day + ' '+ time for time in times]
            for time in times:
                new_location.add_site(time)

In [370]:
for location_name in locations.keys():
    location = locations[location_name]
    print(location.ideal_num_sites)

2
2
2
3
2
2
1
3
2
1


### Create SL objects

In [371]:
for row in sl_availabilities.index.tolist():
    sl_name = sl_availabilities.loc[row, 'Name']
    sl_drives = sl_availabilities.loc[row, 'Drives']
    sl_drives = {'Yes':True, 'No':False}[sl_drives]
    availabilities = sl_availabilities.loc[row, 'Times Available']
    sl = SiteLeader(sl_name, sl_drives, extract_times(availabilities))
    
#DO NOT RUN MORE THAN ONCE, IF YOU DO IT WILL HAVE A KEYERROR
names_to_site_leaders['Vincent'].name = 'Vincent Lee'
names_to_site_leaders['Vincent Lee'] = names_to_site_leaders['Vincent']
names_to_site_leaders.pop('Vincent')
names_to_site_leaders['rishi'].name = 'Rishi Yang'
names_to_site_leaders['Rishi Yang'] = names_to_site_leaders['rishi']
names_to_site_leaders.pop('rishi')
names_to_site_leaders['Zoe Ko'] = SiteLeader('Zoe Ko', True, ['Tuesday 4:00-5:00PM'])

In [372]:
names_to_site_leaders.keys()

dict_keys(['Jared Puerta', 'Marine Lyden', 'Andrew Shih', 'Surabhi Haniyur', 'Aditya Murali', 'Ashley Kaya', 'Ingrid Altunin', 'Alvie Kam', 'Neil Kamdar', 'Ekansh Agrawal', 'Hannah Chou', 'Gaby Tan', 'Carina Kim', 'Jacob Yim', 'Hailey Adams', 'Kenneth Suhaili', 'Lucy Gleeson', 'Kaitlyn Wang', 'Vincent Lee', 'Rishi Yang', 'Zoe Ko'])

In [373]:
confirmed_site_map = pd.read_excel('Spring 2023 Site Map.xlsx')
all_lst = list(confirmed_site_map['Site Leader'])
print("These are the people who fulfill at least one of the following criteria: \n" + 
"1. Are site leaders but did not fill out the google form \n" +
"2. Whose name is not capitalized \n" + 
"3. Whose name is not written as <FIRST_NAME, LAST_NAME>")
set(all_lst[0:21]).difference(set(names_to_site_leaders.keys()))


These are the people who fulfill at least one of the following criteria: 
1. Are site leaders but did not fill out the google form 
2. Whose name is not capitalized 
3. Whose name is not written as <FIRST_NAME, LAST_NAME>


set()

In [374]:
#---------------Site Leader Assignments-----------------#
""" 
Assume we have each site leader instance along with each potential site created.
We need to figure out which site leader goes to which site. Once this is figured out,
we can reduce the number of "potential sites" to the number of actual sites.

The common principle that we are finding is that we have to go all the way to the leaf before
coming up for air and trying a different possibility. Every time we come up for air, we have
to reverse what we just did.

Either we create another Arrangement class that works with our newly created classes or 
we create a latest_person_added feature + use some lists or something idk?
"""
working_SL_arrangements = []
def display_sites():
    df = pd.DataFrame(columns = ['Day', 'Time', 'District', 'Site', 'Site Leader', 'Drives', 'Site Leader', 'Non-SL Staff Member', 'Site Member 1', 'Site Member 2', 'Site Member 3'])
    for i, site in enumerate(times_to_sites.values()):
        if site.has_site_leader:
            site_member_number = 1
            for member in site.members:
                if member.leads_site:
                    site.loc[i, 'Site Leader'] = member
                elif member.in_staff:
                    site.loc[i, 'Non-SL Staff Member'] = member
                else:
                    site.loc[i, 'Site Member {}'.format(site_member_number)] = member
                    site_member_number += 1
            day, time = site.time.split()
            site.loc[i, 'Day'] = day
            site.loc[i, 'Time'] = time
            site.loc[i, 'District'] = site.district
            site.loc[i, 'Site'] = site.name
    return df
confirmed_site_map = pd.read_excel('Spring 2023 Site Map.xlsx')
def check_SL_matches(person):
    for row in confirmed_site_map.index.tolist():
        if confirmed_site_map.loc[row, 'Site Leader'] == person.name:
            if person.assigned_site.name == confirmed_site_map.loc[row, 'Site']:
                return True
            else:
                return False
    raise Exception()

def check_num_SLs(min_SL = 1, max_SL = 4):
    above_or_equal_min = all([location.num_site_leaders() >= min_SL for location in locations.values()])
    below_or_equal_min = all([location.num_site_leaders() <= max_SL for location in locations.values()])
    return above_or_equal_min and below_or_equal_min

def check_num_staff_or_nonstaff(min_num, max_num, decal_type): #Assumes each site has a SL
    assert decal_type in ('staff', 'nonstaff'), 'Wrong string fed into decal_type argument'
    #Checks number of staff or number of nonstaff members
    if decal_type == 'staff':
        above_or_equal_min = all([site.num_nonstaff() >= min_num for site in times_to_sites.values()])
        below_or_equal_min = all([site.num_nonstaff() <= max_num for site in times_to_sites.values()])
    else:
        above_or_equal_min = all([site.num_staff() >= min_num for site in times_to_sites.values()])
        below_or_equal_min = all([site.num_staff() <= max_num for site in times_to_sites.values()])
    return above_or_equal_min and below_or_equal_min

def check_num_drivers():
    return all([site.has_driver for site in times_to_sites.values()])

def check_sites():
    correct_num_sites = check_num_SLs()
    correct_num_staff = check_num_staff_or_nonstaff(min_staff, max_staff, 'staff') 
    correct_num_nonstaff = check_num_staff_or_nonstaff(min_nonstaff, max_nonstaff, 'nonstaff')
    correct_num_drivers = check_num_drivers()
    return correct_num_drivers and correct_num_staff and correct_num_nonstaff

from math import factorial
def calculate_total_arrangements():
    num_total_sites = len(times_to_sites.values())
    num_site_leaders = len(names_to_site_leaders.keys())
    print("Number of total sites: {}".format(num_total_sites))
    print("Number of site leaders: {}".format(num_site_leaders))
    return factorial(num_total_sites)/factorial(num_total_sites - num_site_leaders)

num_permutations = 0
min_staff, max_staff = 0, 1
min_nonstaff, max_nonstaff = 2, 3
min_sites_per_location, max_sites_per_location = 1, 3
max_SL_arrangements = 10

def initialize_df():
    columns = ['Day', 'Time', 'District', 'Site', 'Site Leader', \
        'Driver', 'Site Member 1', 'Site Member 2', 'Site Member 3', \
        'Site Member 4']
    index = [i for i in range(len(names_to_site_leaders.keys()))]
    df = pd.DataFrame(index = index, columns = columns)
    return df
    
    
def create_df():
    df = initialize_df()
    for i, sl in enumerate(names_to_site_leaders.values()):
        sl_name = sl.name
        df.loc[i, 'Site Leader'] = sl_name
        site = sl.assigned_site
        df.loc[i, 'Site'] = site.name
        df.loc[i, 'District'] = site.district
        day, time = site.time.strip().split()
        df.loc[i, 'Day'] = day
        df.loc[i, 'Time'] = time
        if site.has_driver:
            driver_name = site.get_driver_name()
            df.loc[i, 'Driver'] = driver_name
        if site.num_staff() == 2:
            non_SL_staff_name = site.get_non_SL_staff_name()
            df.loc[i, 'Site Member 1'] = non_SL_staff_name
        if site.num_nonstaff() > 1:
            nonstaff_names = site.get_nonstaff_names()
            for i in range(2, len(nonstaff_names) + 2):
                df.loc[i, 'Site Member ' + str(i)] = nonstaff_names[i-1]
    return df

def find_SL_arrangements(ordered_SL, assigned = []):
    #ordered SL is list of ordered site leaders
    #Go to alive leaf or dead leaf -> base case, no leftover people, each Location has ideal number of sites +- 1, extra sites are fine
    if len(working_SL_arrangements) < max_SL_arrangements:
        if ordered_SL == []:
            if check_num_SLs(min_sites_per_location, max_sites_per_location):
                working_SL_arrangements.append(create_df())
        else:
            unassigned = [person for person in ordered_SL if person not in assigned]
            next_person = unassigned[0]
            for availability in next_person.availabilities:
                try:
                    for site in times_to_sites[availability]:
                        relevant_location = locations[site.name]
                        if not site.has_site_leader and relevant_location.num_site_leaders() + 1 <= max_sites_per_location:
                            site.add_member(next_person)
                            new_unassigned = [person for person in unassigned if person is not next_person]
                            find_SL_arrangements(new_unassigned, assigned + [next_person])
                            site.remove_member(next_person)
                except:
                    raise KeyError('Availability {} is not found in dictionary')

def find_staff_arrangements(ordered_staff, assigned = []):
    if ordered_staff == []:
        if check_num_staff_or_nonstaff(min_staff, max_staff, 'staff'):
            pass #FIXME
    else:
        unassigned = [person for person in ordered_staff if person not in assigned]
        next_person = unassigned[0]
        for availability in next_person.availabilities:
            try:
                for site in times_to_sites[availability]:
                    if site.num_staff() + 1 <= max_staff:
                        if next_person.drives and site.has_driver:
                            pass
                        else:
                            site.add_member(next_person)
                            new_unassigned = [person for person in unassigned if person is not next_person]
                            find_staff_arrangements(new_unassigned, assigned + [next_person])
                            site.remove_member(next_person)
            except:
                raise KeyError('Availability {} is not found in dictionary')
    
def find_nonstaff_arrangements(ordered_nonstaff, assigned = []):
    if ordered_nonstaff == []:
        if check_sites():
            pass #FIXME
    else:
        unassigned = [person for person in ordered_nonstaff if person not in assigned]
        next_person = unassigned[0]
        for availability in next_person.availabilities:
            try:
                for site in times_to_sites[availability]:
                    if site.num_nonstaff() + 1 <= max_nonstaff:
                        if next_person.drives and site.has_driver:
                            pass
                        else:
                            site.add_member(next_person)
                            new_unassigned = [person for person in unassigned if person is not next_person]
                            find_nonstaff_arrangements(new_unassigned, assigned + [next_person])
                            site.remove_member(next_person)
            except:
                raise KeyError('Availability {} is not found in dictionary')

def find_nonSL_arrangements(ordered, assigned = []):
    if ordered == []:
        if check_sites():
            pass #FIXME
    else:
        unassigned = [person for person in ordered if person not in assigned]
        next_person = unassigned[0]
        for availability in next_person.availabilities:
            try:
                for site in times_to_sites[availability]:
                    if next_person.in_staff:
                        if site.num_staff() + 1 <= max_staff:
                            if next_person.drives and site.has_driver:
                                pass
                            else:
                                site.add_member(next_person)
                                new_unassigned = [person for person in unassigned if person is not next_person]
                                find_nonSL_arrangements(new_unassigned, assigned + [next_person])
                                site.remove_member(next_person)
                    else:
                        if site.num_nonstaff() + 1 <= max_nonstaff:
                            if next_person.drives and site.has_driver:
                                pass
                            else:
                                site.add_member(next_person)
                                new_unassigned = [person for person in unassigned if person is not next_person]
                                find_nonSL_arrangements(new_unassigned, assigned + [next_person])
                                site.remove_member(next_person)
            except:
                raise KeyError('Availability {} is not found in dictionary')


In [386]:
ordered_site_leaders = order_by_availabilities(names_to_site_leaders.values())
print([person.name for person in ordered_site_leaders])
# clear_all_sites()
# print("Maximum number of permutations is {}".format(calculate_total_arrangements()))
# find_SL_arrangements(ordered_site_leaders)

['Marine Lyden', 'Andrew Shih', 'Aditya Murali', 'Alvie Kam', 'Zoe Ko', 'Hailey Adams', 'Kaitlyn Wang', 'Vincent Lee', 'Neil Kamdar', 'Ekansh Agrawal', 'Ashley Kaya', 'Gaby Tan', 'Ingrid Altunin', 'Carina Kim', 'Kenneth Suhaili', 'Surabhi Haniyur', 'Rishi Yang', 'Lucy Gleeson', 'Jacob Yim', 'Jared Puerta', 'Hannah Chou']
['Marine Lyden', 'Andrew Shih', 'Aditya Murali', 'Alvie Kam', 'Zoe Ko', 'Hailey Adams', 'Kaitlyn Wang', 'Vincent Lee', 'Neil Kamdar', 'Ekansh Agrawal', 'Ashley Kaya', 'Gaby Tan', 'Ingrid Altunin', 'Carina Kim', 'Kenneth Suhaili', 'Surabhi Haniyur', 'Rishi Yang', 'Lucy Gleeson', 'Jacob Yim', 'Jared Puerta', 'Hannah Chou']


In [376]:
working_SL_arrangements[0]

,Day,Time,District,Site,Site Leader,Driver,Site Member 1,Site Member 2,Site Member 3,Site Member 4
0,Thursday,9:20-10:20AM,WCCUSD,Harding,Jared Puerta,NaN,NaN,NaN,NaN,NaN
1,Wednesday,5:00-6:00PM,Aspire,Maynard,Marine Lyden,NaN,NaN,NaN,NaN,NaN
2,Wednesday,5:00-6:00PM,Aspire,ACA,Andrew Shih,Andrew Shih,NaN,NaN,NaN,NaN
3,Tuesday,3:30-4:30PM,EBAC,Highland Community,Surabhi Haniyur,NaN,NaN,NaN,NaN,NaN
4,Friday,9:20-10:20AM,WCCUSD,Harding,Aditya Murali,NaN,NaN,NaN,NaN,NaN
5,Tuesday,3:30-4:30PM,EBAC,Peralta,Ashley Kaya,NaN,NaN,NaN,NaN,NaN
6,Tuesday,5:00-6:00PM,Aspire,ACA,Ingrid Altunin,NaN,NaN,NaN,NaN,NaN
7,Wednesday,2:00-3:00PM,EBAC,Crocker Highlands,Alvie Kam,Alvie Kam,NaN,NaN,NaN,NaN
8,Wednesday,3:30-4:30PM,EBAC,Highland Community,Neil Kamdar,Neil Kamdar,NaN,NaN,NaN,NaN
9,Thursday,3:30-4:30PM,EBAC,Peralta,Ekansh Agrawal,Ekansh Agrawal,NaN,NaN,NaN,NaN


In [377]:
for location_name in locations.keys():
    print(location_name)

Peralta
Achieve
Crocker Highlands
Highland Community
Harding
Stewart
Fairmont
Maynard
ACA
RTA


# Stage 3: Use Site Leader Arrangements to create dataframe

### Fix Site Map Excel Sheet

In [379]:
#Rename columns
new_column_mappings = {'Site Member': 'Site Member 1', 'Site Member.1' : 'Site Member 2', 'Site Member.2':'Site Member 3', 'Site Member.3' : 'Site Member 4'}
confirmed_site_map = confirmed_site_map.rename(columns = new_column_mappings)
#Assign current sites
clear_all_sites()
def find_site_without_site_leader(day_and_time, site_name):
    potential_sites = times_to_sites[day_and_time]
    site_name_minus_letter = remove_letter_after_site_name(site_name)
    site = [site for site in potential_sites if \
        site.name == site_name_minus_letter and not site.has_site_leader][0]
    return site
def remove_letter_after_site_name(site_name):
    if re.compile(r'([a-z]+[\s])+[a-z]').match(site_name.lower()):
        return site_name[0:-1].strip()
    else:
        return site_name.strip()
for row in range(len(names_to_site_leaders.keys())):
    sl_name = confirmed_site_map.loc[row, 'Site Leader']
    day = confirmed_site_map.loc[row, 'Day']
    time = confirmed_site_map.loc[row, 'Time']
    print(sl_name, day, time)
    day_and_time = day + ' ' + time
    site_name = confirmed_site_map.loc[row, 'Site']
    sl = names_to_site_leaders[sl_name]
    site = find_site_without_site_leader(day_and_time, site_name)
    site.add_member(sl)
    if sl.drives:
        confirmed_site_map.loc[row, 'Driver'] = sl_name

Hannah Chou Tuesday 3:30-4:30PM
Lucy Gleeson Tuesday 3:30-4:30PM
Surabhi Haniyur Tuesday 3:30-4:30PM
Carina Kim Tuesday 3:45-4:45PM
Zoe Ko Tuesday 4:00-5:00PM
Jared Puerta Tuesday 5:00-6:00PM
Alvie Kam Wednesday 2:00-3:00PM
Kaitlyn Wang Wednesday 3:00-4:00PM
Neil Kamdar Wednesday 3:30-4:30PM
Hailey Adams Wednesday 4:00-5:00PM
Andrew Shih Wednesday 5:00-6:00PM
Marine Lyden Wednesday 5:00-6:00PM
Vincent Lee Thursday 10:55-11:55AM
Gaby Tan Thursday 12:00-1:00PM
Ekansh Agrawal Thursday 3:30-4:30PM
Rishi Yang Thursday 3:30-4:30PM
Ashley Kaya Thursday 4:00-5:00PM
Jacob Yim Thursday 4:30-5:30PM
Aditya Murali Friday 9:20-10:20AM
Kenneth Suhaili Friday 1:30-2:30PM
Ingrid Altunin Friday 1:40-2:40PM


In [383]:
confirmed_site_map = confirmed_site_map.loc[0:20]
confirmed_site_map.to_excel('Spring 2023 Site Map.xlsx')

# Stage 4: Get responses from rest of decal & assign them to each site

### Fix Staff Availabilities spreadsheet

In [387]:
staff_availabilities = pd.read_csv('Staff Availability Sp 23.csv', header = 0)
staff_availabilities

,Timestamp,Username,Name,Please select all times you are available to go to site.,"Do you have access to a car that seats 5 people, including yourself, that you can drive to site?",Are you willing and able to rent a Gig/Zipcar/etc. to drive to site?,Have you had a TB test done in the last year?,"If your answer to the previous question was No, have you had a TB test done in the last two years?",Any comments/suggestions for SL coordinators
0,2023/01/25 9:28:20 PM PST,emilyyepez@berkeley.edu,Emily Yepez,Thursday 12-1pm;Thursday 1-2pm;Thursday 1:30-2...,No,No,Yes,Yes,NaN
1,2023/01/25 9:29:52 PM PST,johnnyluu@berkeley.edu,Johnny Luu,Friday 1:30-2:30pm,No,Yes,Yes,Yes,NaN
2,2023/01/25 9:32:44 PM PST,c.juster@berkeley.edu,Charlie Juster,Wednesday 2-3pm;Wednesday 3-4pm;Wednesday 3:30...,No,No,No,No,NaN
3,2023/01/25 9:32:57 PM PST,safaamouline@berkeley.edu,Safaa Mouline,Tuesday 4-5pm;Tuesday 5-6pm;Thursday 4-5pm;Thu...,No,No,No,Yes,NaN
4,2023/01/25 9:33:25 PM PST,nicolexin@berkeley.edu,Nicole Xin,Wednesday 5-6pm,Yes,No,Yes,Yes,my car’s at the fulton house so may be a tad b...
5,2023/01/25 9:36:55 PM PST,psuwardi@berkeley.edu,Patricia Suwardi,Tuesday 4-5pm;Tuesday 5-6pm,No,No,No,Yes,NaN
6,2023/01/25 10:54:38 PM PST,marinelyden@berkeley.edu,Marine,Tuesday 4-5pm;Tuesday 5-6pm,No,No,Yes,Yes,NaN
7,2023/01/25 11:08:27 PM PST,aksharav@berkeley.edu,Akshara Vykunta,Thursday 9:20-10:20am;Thursday 10:55-11:55am,No,No,Yes,Yes,NaN
8,2023/01/26 1:41:18 AM PST,sunaydagli@berkeley.edu,Sunay Dagli,Tuesday 3:30-4:30pm;Tuesday 3:45-4:45pm;Wednes...,No,Yes,Yes,Yes,NaN
9,2023/01/26 10:25:44 AM PST,kraecorpus@berkeley.edu,Kaitlin Corpus,Wednesday 4-5pm;Wednesday 5-6pm;Thursday 1-2pm...,No,No,No,No,NaN
